# **과제2 : 챗봇 만들기2**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기2
    * Vector DB에 데이터 추가하기
    * Retriever, memory, LLM를 연결하기
    * 실행시 이력 DB 생성하고 기록하기
    * test

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [2]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key) 

sk-proj-0nqN2ZeDdjgCtfkJn4HuT3BlbkFJcnZowGo2zJAWuP0bJ3Zl


* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [3]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키' 
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들기**

* 데이터 로딩 
    * 1일차에서 제공한 csv 파일의 구조를 그대로 이용
    * 에이블스쿨 홈페이지 FAQ 데이터 수집(https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056)
        * 모든 질문을 csv 형태로 저장
    * 데이터프레임으로 저장하기

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


# 세션 생성
session = requests.Session()

# 세션을 사용해서 메인 페이지를 요청
response = session.get('https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056')
print(response)

subject = []
QA = []

# 1페이지부터 8페이지까지 이동하며 크롤링
for i in range(1, 8):
    
    # 질문과 답변이 있는 URL에서 GET방식으로 크롤링
    response = session.get(f'https://aivle.kt.co.kr/home/brd/faq/listJson?ctgrCd=&pageIndex={i}')
    
    # 필요한 데이터만 추출
    returnList = response.json()['returnList']
    
    # 각 페이지에서 [구분, 질문 + 답변]으로 리스트에 append
    for item in returnList:
        subject.append(item['ctgrNm'])
        QA.append(item['atclTitle'] + '\n' + BeautifulSoup(item['atclCts'], 'html.parser').get_text())
        

data = {
    '구분': subject,
    'QA': QA
}       

# 데이터프레임 생성
df = pd.DataFrame(data)

# 데이터프레임을 csv파일로 저장
df.to_csv('aivleschool_totalqa.csv', index=False)


<Response [200]>


C:\Users\User\AppData\Local\Temp\ipykernel_29972\4077522380.py:20: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  QA.append(item['atclTitle'] + '\n' + BeautifulSoup(item['atclCts'], 'html.parser').get_text())


* 벡터 데이터베이스
    * 1일차 벡터 데이터베이스를 그대로 이용
        * Embedding 모델 : text-embedding-ada-002
        * DB 경로 : ./db



In [3]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory="./database2",  # 경로 지정(현 위치에서 db 폴더 생성)
                    embedding_function = embeddings  # 임베딩 벡터로 만들 모델 지정
)

* 데이터 입력
    * 기존 입력을 모두 제거하고 추가 사항만 모두 입력
    * meta data로 '구분' 칼럼 값 추가하기

In [4]:
# csv파일 불러오기
data = pd.read_csv('aivleschool_totalqa.csv', encoding='utf-8')

# 데이터 확인
data.head()

,구분,QA
0,모집/선발,1) 최종 학력 또는 전공과 관계없이 지원할 수 있나요? \nKT 에이블스쿨은 정규...
1,모집/선발,2) 35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하...
2,모집/선발,3) 미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다. \n1) 기간...
3,모집/선발,"4) 직장인도 지원할 수 있나요? \nKT 에이블스쿨은 미취업자를 대상으로 하며, ..."
4,모집/선발,5) 아르바이트를 하고 있는데 지원할 수 있나요? \n고용보험에 가입이 되어 있는 ...


In [7]:
# 데이터베이스에 데이터 저장
input_list, metadata = [], []
for d in data.values:
    input_list.append(d[1])
    metadata.append({'category': d[0]})

doc = [Document(page_content = input_list[i], metadata = metadata[i]) for i in range(len(data))]
ind = database.add_documents(doc)

* 입력된 데이터 조회

In [4]:
database.get()

{'ids': ['0371ab1a-eeba-45c5-83c2-4b5a68e394df',
  '08d5a769-56c4-42b2-bb58-b02a30addaf3',
  '08f4b700-c6d2-4191-95c1-2cd56bf81583',
  '0d31846b-c177-45a3-8d6e-280b17d168a5',
  '11a3826f-124e-48e9-b3eb-6a2890631e9d',
  '11aaae05-7dbe-44d9-81af-b9f2195ca955',
  '12dc6711-b4e1-42ad-a7da-5f4b9c46ff62',
  '134d2a55-8c11-46c2-a9da-5d63ba2c7cd8',
  '1458d6e2-a9e1-454d-ac19-1e2bf493b9ab',
  '14aa935c-9a04-4106-8f9b-3485bd32eae9',
  '17abeb51-1ea7-4f03-a100-113436ab5d92',
  '198f8cec-54bf-4dac-b04a-4d27c04ed5a0',
  '19f05cd4-070f-4b3f-b302-d0922ff406d3',
  '1a65b6a7-d2ca-4765-92ea-c56b5f90b4e1',
  '1d725cca-8c1e-4a87-accf-d18138129903',
  '1d9dc11f-42f7-42ba-a85c-83c5043e10b7',
  '20e0e49f-71c0-4400-9fc9-10f79045628a',
  '20fcd042-7006-4572-bf62-d2fc499e1e73',
  '2a4fadd1-9c70-4fda-ae16-ad29afc8164a',
  '2bae630d-084a-41c1-99f3-d1d6719085a0',
  '2bf0cdd9-9b65-4c04-ac99-ff5eda0b13a0',
  '2e7f5706-9790-4a84-b320-5d3cfa2f84c7',
  '31aae2e1-1807-4c13-9f9c-0da70268b9bd',
  '337a1166-9fbe-4b1f-998c-

## **3.RAG+LLM모델**

* 모델 : ConversationalRetrievalChain
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정
    * memory 사용
* 요구사항
    * 질문 history 관리를 위한 이력 저장 DB 생성
        * DB 명 : db_chatlog
        * 테이블 명 : history
            * id INTEGER PRIMARY KEY : 이렇게 설정하면 자동증가 값으로 채워짐
            * datetime TEXT : 질문시점 yyyy-mm-dd hh:mi:ss
            * query TEXT : 질문
            * sim1 REAL : 첫번째 문서의 유사도 점수
            * sim2 REAL : 두번째 문서의 유사도 점수
            * sim3 REAL : 세번째 문서의 유사도 점수
            * answer TEXT : 답변
        * 유사도 점수는 similarity_search_with_score 메서드를 이용해서 저장해야 함
        * 질문과 답변이 진행될 때마다 history 테이블에 데이터 입력

* 관리용 DB, 테이블 생성

In [5]:
path = './db_chatlog.db'

# DB 연결
conn = sqlite3.connect(path)

# 커서 객체 생성
cursor = conn.cursor()

# history 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS history (
    id INTEGER PRIMARY KEY,
    datetime TEXT NOT NULL,
    query TEXT NOT NULL,
    sim1 REAL NOT NULL,
    sim2 REAL NOT NULL,
    sim3 REAL NOT NULL,
    answer TEXT NOT NULL
)
''')

# 변경사항 커밋 (저장)
conn.commit()

# 연결 종료
conn.close()

* 모델 선언

In [6]:
k = 3

# retriever 선언
retriever = database.as_retriever(search_kwargs={"k": k})

# ChatOpenAI 선언
chat = ChatOpenAI(model="gpt-3.5-turbo")

# 메모리 초기화
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer",
                                  return_messages=True)
# RetrievalQA 선언
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory,
                                           return_source_documents=True,  output_key="answer")



* 모델 사용 및 이력 확인

In [19]:
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer",
                                  return_messages=True)

qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory,
                                           return_source_documents=True,  output_key="answer")
while True:
    
    # DB 연결
    conn = sqlite3.connect(path)
    
    # 명령어 옵션 입력
    cmd = int(input('명령어를 입력하세요. \n [새 대화]: 1 / [기존 대화 유지]: 2 / [프로그램 종료]: 3'))
    
    if cmd == 1 or cmd == 2:
        
        # 질문
        query = input('질문을 입력해주세요.')
        
        if cmd == 1:
            
            # 메모리 초기화
            memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer",
                                  return_messages=True)
            qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory,
                                           return_source_documents=True,  output_key="answer")
            print('메모리 초기화 됨')
       
        # 현재 시간
        dt = datetime.now()
        dt = dt.strftime('%Y-%m-%d %H:%M:%S')

        # 유사성 점수
        sim = []
        result_sim = database.similarity_search_with_score(query=query, k=k)
        for r in result_sim:
            sim.append(round(r[1], 5))

        # 답변
        result = qa(query)
        answer = result['answer']
        print(answer, flush=True)
        print(memory.load_memory_variables({})['chat_history'])
        print('='*70, flush=True)
        
        # 로그 저장
        log = pd.DataFrame({'datetime': [dt], 'query': [query], 'sim1': [sim[0]], 'sim2': [sim[1]], 'sim3': [sim[2]], 'answer': [answer]})
        log.to_sql('history', conn, if_exists='append', index=False)
        
        
    elif cmd == 3: 
        
        # DB 연결 종료
        conn.close()
        break

   

메모리 초기화 됨
안녕하세요! 궁금하신 내용이 있으시면 언제든지 물어보세요. 답변 도와드릴게요. :)
[HumanMessage(content='ㅎㅇ'), AIMessage(content='안녕하세요! 궁금하신 내용이 있으시면 언제든지 물어보세요. 답변 도와드릴게요. :)')]
제가 알기로는 "에이블"에 대한 정보가 없습니다. 추가 정보가 필요하시면 알려주세요.
[HumanMessage(content='ㅎㅇ'), AIMessage(content='안녕하세요! 궁금하신 내용이 있으시면 언제든지 물어보세요. 답변 도와드릴게요. :)'), HumanMessage(content='에이블은 어때'), AIMessage(content='제가 알기로는 "에이블"에 대한 정보가 없습니다. 추가 정보가 필요하시면 알려주세요.')]


In [20]:
# DB 조회
conn = sqlite3.connect(path)

df = pd.read_sql('SELECT * FROM history', conn)
display(df)

conn.close()

,id,datetime,query,sim1,sim2,sim3,answer
0,1,2024-06-04 14:09:00,에이블스쿨이 뭐야?,0.35384,0.35384,0.36587,"에이블스쿨은 KT와 정부가 함께 하는 프로그램으로, 미래인재육성과 청년 IT/SW/..."
1,2,2024-06-04 14:09:18,그럼 KT로 취업할 수 있어?,0.23383,0.23383,0.28435,"KT 에이블스쿨은 미취업자를 대상으로 하고 있으며, 교육 시작일 기준 재직자는 지원..."
2,3,2024-06-04 14:09:37,국민취업제도에 대해서도 궁금해,0.24076,0.24076,0.26745,국민취업제도에 대해 궁금한 내용이 있으시면 자세히 알려드릴 수 있습니다. 어떤 정보...
3,4,2024-06-04 14:10:05,에이블스쿨하고 국민취업제도하고 병행할 수 있어?,0.25058,0.25058,0.26480,KT 에이블스쿨 교육에 참여하시면서 국민취업지원제도를 병행할 수 있습니다. 국민취업...
4,5,2024-06-04 14:10:43,언제까지 발급해야돼?,0.34846,0.34846,0.34913,국민내일배움카드를 발급받는 기한은 없습니다. 발급을 원할 때 신청하실 수 있습니다.
5,6,2024-06-04 14:15:16,에이블스쿨에서 노트북은 지원해줘?,0.23385,0.23385,0.31072,"네, 에이블스쿨은 교육기간 동안에 노트북을 무료로 지원해주고 있습니다. 교육에 필요..."
6,7,2024-06-04 14:15:38,그럼 그거 사양은 어떻게 돼?,0.38098,0.38098,0.42307,교육기간 동안 제공되는 노트북의 사양은 다음과 같습니다:\n- CPU: I5\n- ...
7,8,2024-06-04 14:15:57,만약 노트북이 사용하다가 고장나면?,0.26505,0.26505,0.31256,"고장이 발생한 경우에는 담당자나 관리자에게 문의하여 상황을 보고하고, 필요한 조치를..."
8,9,2024-06-04 14:16:40,에이블스쿨은 교육비가 얼마야?,0.28097,0.28097,0.28386,제가 알기로는 에이블스쿨의 교육비는 무료로 제공되는 것으로 알고 있습니다. 그러나 ...
9,10,2024-06-04 14:17:10,에이블스쿨은 어디서 진행돼?,0.31455,0.31455,0.32126,"KT 에이블스쿨은 온라인으로 진행되며, 인터넷을 통해 수강할 수 있습니다."
